# XAI for Security Tutorial Part 2

#### Run these cells to mount the notebook to the drive folder and set the correct paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
repo_path = "/content/drive/MyDrive/AISEC-SummerSchool-2025/XAI for Security/drebin_xai"
data_path = "/content/drive/MyDrive/AISEC-SummerSchool-2025/XAI for Security/drebin_xai/data"
sys.path.append(f"{repo_path}/src")


#### Run these cells to install and load necessary packages

In [ ]:
# Install model and training environment
!pip install pytorch_lightning

In [ ]:
# Load necesarry packages
import json
import pandas as pd
import numpy as np
import zipfile
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torchmetrics.classification import Accuracy
from pathlib import Path
from typing import List, Dict
from sklearn.datasets import load_svmlight_file

# 2 Gradient-based explanations

In this section we will implement two approaches to use gradients to explain a machine learning method.

### 2.1 Vanilla Gradient

For vanilla gradients, the gradients of a model should be calculated w.r.t. to an input instance and the corresponding model output. This is not so different from the warm-up task, as a machine learning model can also be considered a large function.

### 2.2 Gradient x Input

In the previous approach, only the gradients of the model prediction in relation to the input were analyzed. This type of explanation can also be interpreted as sensitivity. They reflect how the prediction responds to infinitesimal changes in the inputs. However, the values of the input features have not been taken into account so far. As the name of the method suggests, the values of an input instance are multiplied by the gradients.

In [ ]:
class GradientExplainer:
  def __init__(self, model):
    self.model = model
    self.model.eval()

  def prep_input(input):
      # Ensure that input is tensor
      if isinstance(input, np.ndarray):
        input = torch.from_numpy(input)
        input = input.float()
      input = input.unsqueeze(0).clone().detach()
      return input

  # TODO: Implement vanilla gradients explanation approach
  def vanilla_gradient(self, input, target=None):
    input = GradientExplainer.prep_input(input)

    # Your code here
    input.requires_grad = True
    # Forward-pass
    output = self.model(input)

    # Remove batch dimension
    output = output.squeeze(0)
    # Determine scalar for backprop
    if output.dim() == 0:
        logit = output
    else:
        predicted_class = output.argmax().item()
        logit = output[predicted_class]

    # Your code here
    # Compute gradient starting from logit
    self.model.zero_grad()
    logit.backward()

    return input.grad[0].detach().cpu().numpy().squeeze()

  # TODO: Implement Gradient X Input explanation
  def gradient_x_input(self, input, target=None):
    # Your code here
    grad = self.vanilla_gradient(input)
    gradients = input * grad
    return gradients.squeeze()

In [ ]:
#@title Solution Vanilla Gradient
def vanilla_gradient(self, input, target=None):
    input = GradientExplainer.prep_input(input)

    # Your code here
    input.requires_grad = True
    # Forward-pass
    output = self.model(input)

    # Remove batch dimension
    output = output.squeeze(0)
    # Determine scalar for backprop
    if output.dim() == 0:
        logit = output
    else:
        predicted_class = output.argmax().item()
        logit = output[predicted_class]

    # Your code here
    # Compute gradient starting from logit
    self.model.zero_grad()
    logit.backward()

    return input.grad[0].detach().cpu().numpy().squeeze()

In [ ]:
#@title Solution Gradient X Input
def gradient_x_input(self, input, target=None):
    grad = self.vanilla_gradient(input)
    gradients = input * grad
    return gradients.squeeze()

### 2.3 Difference between Vanilla Gradient and Gradient x Input explanation

As mentioned earlier, vanilla gradients and the gradient × input method differ conceptually. Vanilla gradients measure the sensitivity of the model’s prediction with respect to each input feature, capturing how much a small change in a feature, e.g. a pixel, would affect the output. In contrast, the gradient × input approach also considers the magnitude of the input feature itself, providing a form of feature attribution: it indicates how much the actual value of a feature contributes to the model’s prediction.

In the following notebook you will apply the implementations form above to two examples that illustrate the difference.

**Example 1:**

Lets assume we have a simple model:

$f(x) = 2 * x_1 + 0.1 * x_2 $, and an input $ x = [1, 10] $

Use your own implementations from above to compute the different gradient explanations and compare the results.

In [ ]:
# Input
x = torch.tensor([1., 10.])

# Define toy model that represents the function f(x) = 2*x_1+0.1*x_2
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Linear layer without bias that takes two values and outputs one input
        self.linear = nn.Linear(2, 1, bias=False)
        # Manually set weights to match f(x1, x2) = 2*x1 + 0.1*x2
        with torch.no_grad():
            self.linear.weight.copy_(torch.tensor([[2.0, 0.1]]))

    def forward(self, x):
        return self.linear(x)

# Create model
model = SimpleModel()


# TODO: compute vanilla gradient and gradient x input for the SimpleModel with regards to x
# Your solution here
explainer = GradientExplainer(model)
vanilla_gradient = explainer.vanilla_gradient(x)
gradient_x_input = explainer.gradient_x_input(x)

print(f"Vanilla Gradient: {vanilla_gradient}")
print(f"Gradient x Input: {gradient_x_input}")


In [ ]:
#@title Solution
explainer = GradientExplainer(model)
vanilla_gradient = explainer.vanilla_gradient(x)
gradient_x_input = explainer.gradient_x_input(x)

print(f"Vanilla Gradient: {vanilla_gradient.numpy()}")
print(f"Gradient x Input: {gradient_x_input.numpy()}")

**Example 2**

In the second example we will analyze the differences in a more sophisticated use case, i.e. classification of handwritten digits. In the following notebook will:

1. Load MNIST sample data and perform data preparation steps
2. Implement a simple CNN model and training loop
3. Train the model for the classification task on the training split and provide validation performance
4. Load test instance to explain model predictions

In [ ]:
from mnist_utils import *

# Load data
mnist_path = f"{data_path}/mnist/mnist_train_small.csv"
train_loader, val_loader = load_and_prep_data(mnist_path)

# Build model
model = LitMNIST_CNN(lr=0.001)

# Train model
fit_model(model, train_loader, val_loader)


In [ ]:
# Load test data to explain
test_path = f"{data_path}/mnist/mnist_test.csv"
test_dat = load_test_data(test_path)

In [ ]:
# Your code here
# Initialize GradientExplainer object with trained model and generate
# vanilla gradient and gradient_x_input explanation of the first instace of the test data

explainer = GradientExplainer(model)
input, _ = test_dat[0]
vanilla_gradient = explainer.vanilla_gradient(input)
grad_x_input = explainer.gradient_x_input(input)

plot_gradients(input, vanilla_gradient, grad_x_input)

# 3. Saliency Maps for DeepDrebin (Gradients & Gradients × Input)

This notebook:
1. Loads a single **bundle** file (model + selector + selected feature names).
2. Loads a filtered **LIBSVM** dataset (labels `-1/+1` → `{0,1}`).
3. Splits into **train/val/test** using the same seed/splits stored in the bundle (for comparability).
4. Transforms with the bundled **SelectKBest** into the selected-feature space.
5. Picks **N malicious** samples (by label) and computes saliency with:
   - **Gradients**: $|\frac{\partial (\text{logit})}{\partial x}|$
   - **Gradients × Input**: $|x \cdot \frac{\partial (logit)}{\partial x}|$
6. Prints **top-K feature names** with scores and (optionally) saves a CSV.

In [ ]:
from drebin_utils import *

In [ ]:
# Set configs
DATA_PATH = f"{data_path}/drebin"
BUNDLE_PATH = os.path.join(DATA_PATH, "drebin_bundle.pt")
LIBSVM_DATA = os.path.join(DATA_PATH, "drebin_filtered.libsvm.gz")
NUM_MALWARE = 5
TOP_K = 10
DEVICE = "cpu"

In [ ]:
# Build model and load data
model, bundle = build_model(BUNDLE_PATH, DEVICE)
X, y = load_libsvm(LIBSVM_DATA, bundle["selector"], bundle["model_hparams"])

In [ ]:
# Pick instances to explain
picked_indices, probs_all = pick_malicious_indices(X, y, model, num=NUM_MALWARE)

print(f"Selected {len(picked_indices)} malicious samples:")
for r, i in enumerate(picked_indices, 1):
    gt = f", y={int(y[i])}" if y is not None else ""
    print(f"{r:2d}. idx={i:>6}  prob_mal={probs_all[i]:.4f}{gt}")

In [ ]:
def explain_sample(bundle, i: int, top_k: int = 10) -> dict:
    """
    Explain a single sample using Gradients and Input×Gradients.
    Shows only positive (malicious) contributions, sorted descending.
    Renders a heatmap for each method with a red→blue colorbar (cells in shades of red).
    """
    x = X[i]

    # Your code here
    #
    explainer = GradientExplainer(model)
    grad = explainer.vanilla_gradient(x)
    ixg = explainer.gradient_x_input(x)
    #grad = compute_gradients(model, x)
    #ixg  = compute_input_x_gradients(model, x)

    # Keep *only* positive contributions and take top-k
    grad_idx = topk_positive(grad, top_k)
    ixg_idx  = topk_positive(ixg,  top_k)

    # Build lists sorted by descending score (most malicious first)
    grad_scores = np.array([grad[j] for j in grad_idx], dtype=float)
    grad_names  = [fname(bundle["selected_names"], j) for j in grad_idx]

    ixg_scores  = np.array([ixg[j]  for j in ixg_idx], dtype=float)
    ixg_names   = [fname(bundle["selected_names"], j) for j in ixg_idx]

    # Header
    print(f"\n=== Sample {i} ===")
    if y is not None:
        print(f"Ground truth label: {int(y[i])} (1=malware)")
    p = float(predict_proba(model, X[i:i+1])[0])
    print(f"Predicted malware probability: {p:.4f}")

    # Heatmaps: most relevant on top (already sorted)
    if len(grad_scores):
        plot_malicious_heatmap(grad_scores, grad_names, "Gradients (malicious)")
    else:
        print("No positive (malicious) gradient contributions found.")

    if len(ixg_scores):
        plot_malicious_heatmap(ixg_scores, ixg_names, "Input × Gradients (malicious)")
    else:
        print("No positive (malicious) Input×Gradients contributions found.")

    return {
        "sample_index": i,
        "prob_malware": p,
        "top_gradients": [
            {"index": int(j), "name": fname(bundle["selected_names"], j), "score": float(grad[j])}
            for j in grad_idx
        ],
        "top_input_x_gradients": [
            {"index": int(j), "name": fname(bundle["selected_names"], j), "score": float(ixg[j])}
            for j in ixg_idx
        ],
    }

# Re-run explanations for the picked malicious samples
explanations = [explain_sample(bundle, i, top_k=TOP_K) for i in picked_indices]